In [1]:
!git clone https://github.com/docongminh/Classification-SIFT-SVM.git

Cloning into 'Classification-SIFT-SVM'...
remote: Enumerating objects: 1912, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1912 (delta 1), reused 2 (delta 0), pack-reused 1907
Receiving objects: 100% (1912/1912), 31.82 MiB | 47.36 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [2]:
%cd /content/Classification-SIFT-SVM

/content/Classification-SIFT-SVM


In [4]:
!python build_model.py --help


usage: build_model.py [-h] -dt INPUTDATA

optional arguments:
  -h, --help            show this help message and exit
  -dt INPUTDATA, --inputdata INPUTDATA
                        path to data training


In [4]:
!python build_model.py -dt '/content/drive/MyDrive/images/recognition'

/content/drive/MyDrive/images/recognition
[ WARN:0@90.202] global /io/opencv_contrib/modules/xfeatures2d/misc/python/shadow_sift.hpp (13) SIFT_create DEPRECATED: cv.xfeatures2d.SIFT_create() is deprecated due SIFT tranfer to the main repository. https://github.com/opencv/opencv/issues/16736
tcmalloc: large alloc 5427625984 bytes == 0x2f0cc0000 @  0x7fb7d784b1e7 0x7fb7d459d0ce 0x7fb7d45f9715 0x7fb7d45f9d1b 0x7fb7d469b41b 0x5936cc 0x548c51 0x5127f1 0x549e0e 0x4bca8a 0x5134a6 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x511e2c 0x593dd7 0x5118f8 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7fb7d7448c87 0x5b621a
tcmalloc: large alloc 2713812992 bytes == 0x434ea8000 @  0x7fb7d784b1e7 0x7fb7d459d0ce 0x7fb7d45f9715 0x7fb7d46989d6 0x4bc4ab 0x7fb7d45de944 0x59371f 0x515244 0x549576 0x593fce 0x548ae9 0x51566f 0x549e0e 0x4bca8a 0x5134a6 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x511e2c 0x593dd7 0x5118f8 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64fa

In [9]:
!python gridSearchCV.py 

Traceback (most recent call last):
  File "gridSearchCV.py", line 12, in <module>
    ['clf', SVC(random_state = 0, probability = True)]
TypeError: __init__() takes 2 positional arguments but 3 were given


In [13]:
import cv2
import os
import glob


def imread_img(path_image):

	return cv2.imread(path_image)

def label_image(path_to_data):

	list_label = os.listdir(path_to_data)

	label2id = {}
	for id_, value in enumerate(list_label):
		label2id[value] = id_

	data_train = []
	data_label = []

	for label in list_label:
		for img in glob.glob(path_to_data+"/" + label + '/*'):
			image = imread_img(img)
			label_image = label2id[label]

			data_train.append(image)
			data_label.append(label_image)

	return data_train, data_label, label2id

# data_train, data_label, label2id = label_image('data/')
#
# print(data_train[0])
# cv2.imwrite('test.jpg', data_train[0])
# print(data_label[0])
# print(label2id)

In [14]:
import cv2
from sklearn.cluster import KMeans
import pickle
from scipy.spatial.distance import cdist
import os
import numpy as np


def extract_sift_features(list_image):

    image_descriptors = []
    sift = cv2.xfeatures2d.SIFT_create()
    for image in list_image:
        _, descriptor = sift.detectAndCompute(image, None)
        image_descriptors.append(descriptor)

    return image_descriptors

def kmean_bow(all_descriptors, num_cluster):
    bow_dict = []

    kmeans = KMeans(n_clusters = num_cluster)
    kmeans.fit(all_descriptors)

    bow_dict = kmeans.cluster_centers_

    if not os.path.isfile('bow_dictionary.pkl'):
        pickle.dump(bow_dict, open('bow_dictionary.pkl', 'wb'))

    return bow_dict

def create_feature_bow(image_descriptors, BoW, num_cluster):

    X_features = []

    for i in range(len(image_descriptors)):
        features = np.array([0] * num_cluster)

        if image_descriptors[i] is not None:
            distance = cdist(image_descriptors[i], BoW)

            argmin = np.argmin(distance, axis = 1)

            for j in argmin:
                features[j] += 1
        X_features.append(features)

    return X_features


In [ ]:
https://www.kaggle.com/code/pranathichunduru/svm-for-multiclass-classification

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
import sklearn
from data_loader import label_image
from sift_extractors import create_feature_bow, extract_sift_features, kmean_bow
import pickle
import argparse
import os

#arg = argparse.ArgumentParser()
#arg.add_argument("-dt", "--inputdata", required = True, help = "path to data training")
#args = vars(arg.parse_args())

path = "/content/drive/MyDrive/images/recognition"
print(path)
data_train, label, label2id = label_image(path)

image_desctiptors = extract_sift_features(data_train)

all_descriptors = []
for descriptor in image_desctiptors:
    if descriptor is not None:
        for des in descriptor:
            all_descriptors.append(des)

num_cluster = 18
BoW = kmean_bow(all_descriptors, num_cluster)
# if os.path.isfile('bow_dictionary.pkl'):
#     BoW = kmean_bow(all_descriptors, num_cluster)
# else:
#     BoW = pickle.load(open('bow_dictionary.pkl', 'rb'))

X_features = create_feature_bow(image_desctiptors, BoW, num_cluster)





X_train, X_test, Y_train, Y_test = train_test_split(X_features, label, test_size = 0.30, random_state = None)

/content/drive/MyDrive/images/recognition


In [ ]:

########DONOT START

from sklearn.model_selection import train_test_split, GridSearchCV
import sklearn
from data_loader import label_image
from sift_extractors import create_feature_bow, extract_sift_features, kmean_bow
import pickle
import argparse
import os

#arg = argparse.ArgumentParser()
#arg.add_argument("-dt", "--inputdata", required = True, help = "path to data training")
#args = vars(arg.parse_args())

path = "/content/drive/MyDrive/images/recognition"
print(path)
data_train, label, label2id = label_image(path)

image_desctiptors = extract_sift_features(data_train)

all_descriptors = []
for descriptor in image_desctiptors:
    if descriptor is not None:
        for des in descriptor:
            all_descriptors.append(des)

num_cluster = 18     #60
BoW = kmean_bow(all_descriptors, num_cluster)
# if os.path.isfile('bow_dictionary.pkl'):
#     BoW = kmean_bow(all_descriptors, num_cluster)
# else:
#     BoW = pickle.load(open('bow_dictionary.pkl', 'rb'))

X_features = create_feature_bow(image_desctiptors, BoW, num_cluster)





X_train, X_test, Y_train, Y_test = train_test_split(X_features, label, test_size = 0.30, random_state = None)
model_svm = sklearn.svm.SVC(C = 18, random_state = 0)
# parameters = [
#     {'C': [20, 25, 30, 35, 40, 45]}

# ]
# grid_model = GridSearchCV(
#     estimator = model_svm,
#     parameters = parameters,
#     cv = 10
#
# )
# grid_model.fit(X_train, Y_train)

model_svm.fit(X_train, Y_train)
filename = 'svm_model.sav'
pickle.dump(model_svm, open(filename, 'wb'))
print("score on training set params: ", model_svm.score(X_train, Y_train))
print("score on testing set params: ", model_svm.score(X_test, Y_test))

In [17]:
#Libraries to Build Ensemble Model : Random Forest Classifier 
# Create the parameter grid based on the results of random search 
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

In [18]:
# Performing CV to tune parameters for best SVM fit 
svm_model = GridSearchCV(SVC(), params_grid, cv=5)
svm_model.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'kernel': ['linear']}])

In [19]:
# View the accuracy score
print('Best score for training data:', svm_model.best_score_,"\n") 

# View the best parameters for the model found using grid search
print('Best C:',svm_model.best_estimator_.C,"\n") 
print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")

final_model = svm_model.best_estimator_
Y_pred = final_model.predict(X_test)
pred_prob = final_model.predict_proba(X_test)
#Y_pred_label = list(encoder.inverse_transform(Y_pred))

Best score for training data: 0.618144208037825 

Best C: 10 

Best Kernel: rbf 

Best Gamma: 0.0001 



In [20]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# Making the Confusion Matrix
#print(pd.crosstab(Y_test_label, Y_pred_label, rownames=['Actual Activity'], colnames=['Predicted Activity']))
print(confusion_matrix(Y_test,Y_pred))
print("\n")
print(classification_report(Y_test,Y_pred))

print("Training set score for SVM: %f" % final_model.score(X_train , Y_train))
print("Testing  set score for SVM: %f" % final_model.score(X_test  , Y_test ))

svm_model.score

[[58  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0]
 [12 46  1  0  0  2  1  0  0  4  0  0  0  0  0  0  1  1]
 [ 6  0 32  0  0  0  1  0  1  1  0  0  0  0  0  2  1  0]
 [ 8  1  0 22  0  1  2  0  0  3  3  1  0  0  0  0  3  3]
 [ 3  0  1  1 26  1  1  0  1  1  5  0  3  0  0  1  2  0]
 [ 5  0  1  0  1 28  6  1  0  2  2  0  1  1  0  1  0  0]
 [ 7  0  0  0  2  0 28  0  1  0  3  0  0  0  0  0  0  0]
 [ 8  0  0  1  0  2 13 33  0  1  1  1  0  0  0  0  1  0]
 [12  0  0  0  0  0  0  0 33  1  1  0  0  0  0  0  0  0]
 [ 8  0  0  0  1  0  3  0  0 28  0  2  2  0  1  0  0  1]
 [ 2  0  2  0  0  0 12  0  0  0 26  1  0  0  1  0  1  0]
 [ 4  0  0  0  0  0  3  0  0  7  0 34  0  0  0  0  5  0]
 [ 3  0  0  0  2  1  0  0  0  3  0  0 28  0  2  2  0  0]
 [ 4  0  0  2  1  0 11  1  0  2  3  0  1 30  0  1  1  0]
 [ 7  0  0  0  1  2  1  0  0  2  0  0  2  1 38  0  0  0]
 [ 3  0  0  0  0  1  6  0  1  0  1  0  0  0  0 40  1  1]
 [13  0  0  0  0  0  1  0  0  2  1  1  0  0  0  0 37  1]
 [ 6  0  0  0  0  0  2  0  0  2

<bound method BaseSearchCV.score of GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'kernel': ['linear']}])>

In [ ]:
import seaborn as sns
from sklearn.utils.multiclass import unique_labels
import pandas as pd
#heat map
def plot(y_true, y_pred):
  labels=unique_labels(Y_test)
  column=[f'Predicted {label}' for label in labels]
  indices=[f'Actual {label}' for label in labels]
  table=pd.DataFrame(confusion_matrix(Y_test,Y_pred), columns=column, index=indices)

  return sns.heatmap(table, annot=True, fmt='d', cmap='viridis')

In [ ]:
#The average precision score in multi-label settings


from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
n_classes=18
# For each class
precision = dict()
recall = dict()
average_precision = dict()
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(Y_test[:, i], pred_prob[:, i])
    average_precision[i] = average_precision_score(Y_test[:, i], pred_prob[:, i])

# A "micro-average": quantifying score on all classes jointly
precision["micro"], recall["micro"], _ = precision_recall_curve(
    Y_test.ravel(), pred_prob.ravel()
)
average_precision["micro"] = average_precision_score(Y_test, pred_prob, average="micro")

In [ ]:
#Plot the micro-averaged Precision-Recall curve
#Average precision (AP) summarizes such a plot as the weighted mean of precisions achieved at each threshold, with the increase in recall from the previous threshold used as the weight
from sklearn.metrics import PrecisionRecallDisplay
display = PrecisionRecallDisplay(
    recall=recall["micro"],
    precision=precision["micro"],
    average_precision=average_precision["micro"],
)
display.plot()
_ = display.ax_.set_title("Micro-averaged over all classes")

In [ ]:
#Plot Precision-Recall curve for each class and iso-f1 curves

import matplotlib.pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay
from itertools import cycle

# setup plot details
colors = cycle(["navy", "turquoise", "darkorange", "cornflowerblue", "teal"])

_, ax = plt.subplots(figsize=(7, 8))

f_scores = np.linspace(0.2, 0.8, num=4)
lines, labels = [], []
for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    (l,) = plt.plot(x[y >= 0], y[y >= 0], color="pink", alpha=0.2)
    plt.annotate("f1={0:0.1f}".format(f_score), xy=(0.9, y[45] + 0.02))

display = PrecisionRecallDisplay(
    recall=recall["micro"],
    precision=precision["micro"],
    average_precision=average_precision["micro"],
)
display.plot(ax=ax, name="Micro-average precision-recall", color="gold")

for i, color in zip(range(n_classes), colors):
    display = PrecisionRecallDisplay(
        recall=recall[i],
        precision=precision[i],
        average_precision=average_precision[i],
    )
    display.plot(ax=ax, name=f"Precision-recall for class {i}", color=color)

# add the legend for the iso-f1 curves
handles, labels = display.ax_.get_legend_handles_labels()
handles.extend([l])
labels.extend(["iso-f1 curves"])
# set the legend and the axes
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.legend(handles=handles, labels=labels, loc="best")
ax.set_title("Extension of Precision-Recall curve to multi-class")

plt.show()